In [1]:
import torch
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid

# Load a sample dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]

class SimpleGCN(torch.nn.Module):
    def __init__(self, num_node_features, num_classes):
        super().__init__()
        # Use num_node_features and num_classes as parameters
        self.conv1 = GCNConv(num_node_features, 4)  # 1st layer: in_dim->4
        self.conv2 = GCNConv(4, num_classes)     # 2nd layer: 4->num_classes

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))  # aggregate + activate
        x = self.conv2(x, edge_index)          # second layer
        return F.log_softmax(x, dim=1)

model = SimpleGCN(data.num_node_features, dataset.num_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Training loop (simplified)
for epoch in range(50):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

# Evaluate accuracy on test nodes
model.eval()
pred = model(data).argmax(dim=1)
acc = (pred[data.test_mask] == data.y[data.test_mask]).float().mean()
print(f'GCN Test Accuracy: {acc:.2f}')

GCN Test Accuracy: 0.73
